# MLOps Zoomcamp: Intro
### Homework

In [59]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_squared_error

In [60]:
df_train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_test = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [61]:
print(df_train.shape[1])

19


#### Q2. Computing duration

In [62]:
def compute_duration(df):
    df = df.copy()
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)

    return df

In [63]:
df_train = compute_duration(df_train)
df_test = compute_duration(df_test)

In [64]:
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [65]:
print(df_train.duration.std().round(2))

42.59


#### Q3. Dropping outliers

In [66]:
def drop_outliers(df):
    df = df.loc[(df.duration >= 1) & (df.duration <= 60), :]
    return df

In [67]:
print(df_train.shape[0])

3066766


In [68]:
df_train = drop_outliers(df_train)
df_test = drop_outliers(df_test)

In [69]:
print(df_train.shape[0])

3009173


In [70]:
round(3009173 / 3066766, 2)

0.98

#### Q4. One-hot encoding

In [72]:
def one_hot_encoding(df, features, encoder=None):
    if not encoder:
        encoder = OneHotEncoder(dtype=int, handle_unknown="infrequent_if_exist")
        encoder.fit(df[features])

    X = encoder.transform(df[features])

    return X, encoder

In [73]:
df_train[['PULocationID', 'DOLocationID']] = df_train[['PULocationID', 'DOLocationID']].astype(str)
df_test[['PULocationID', 'DOLocationID']] = df_test[['PULocationID', 'DOLocationID']].astype(str)

In [74]:
X_train, encoder = one_hot_encoding(df_train, ['PULocationID', 'DOLocationID'])
X_test, _ = one_hot_encoding(df_test, ['PULocationID', 'DOLocationID'], encoder)

In [75]:
target = 'duration'
y_train = df_train[target]
y_test = df_test[target]

#### Q5. Training a model

In [76]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [77]:
y_pred = model.predict(X_train)
root_mean_squared_error(y_train, y_pred)

7.649262103804537

#### Q6. Evaluating the model

In [78]:
y_pred = model.predict(X_test)
root_mean_squared_error(y_test, y_pred)

7.811815054872749